#RAG from Scratch

In [1]:
!pip install -q --upgrade langchain langchain-community langchain-openai
!pip install -q faiss-cpu pypdf tiktoken
!pip uninstall -y transformers
!pip install -q transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.0/331.0 kB 16.1 MB/s eta 0:00:00
Found existing installation: transformers 5.0.0
Uninstalling transformers-5.0.0:
  Successfully uninstalled transformers-5.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.

STEP 1: Data Ingestion

In [5]:
from google.colab import files
uploaded = files.upload()

Saving WINSEM2025-26_VL_CSI2007_00100_ETH_2026-01-23_Hamming-Code.pdf to WINSEM2025-26_VL_CSI2007_00100_ETH_2026-01-23_Hamming-Code.pdf


In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("WINSEM2025-26_VL_CSI2007_00100_ETH_2026-01-23_Hamming-Code.pdf")
documents = loader.load()
print(documents)

[Document(metadata={'producer': 'macOS Version 12.2.1 (Build 21D62) Quartz PDFContext', 'creator': 'Pages', 'creationdate': "D:20260123041704Z00'00'", 'title': 'Hamming Code 2', 'moddate': "D:20260123041704Z00'00'", 'source': 'WINSEM2025-26_VL_CSI2007_00100_ETH_2026-01-23_Hamming-Code.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='Hamming Code in Computer Network \nHamming code is a set of error-correction codes that can be used to detect \nand correct the errors that can occur when the data is moved or stored \nfrom the sender to the receiver. It is technique developed by R.W. \nHamming for error correction. \nRedundant bits – \nRedundant bits are extra binary bits that are generated and added to the \ninformation-carrying bits of data transfer to ensure that no bits were lost \nduring the data transfer. \nThe number of redundant bits can be calculated using the following formula: \n 2^r ≥ m + r + 1 \n where, r = redundant bit, m = data bit\nSuppose the number of

STEP 2: Chunking

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(documents)

print(len(chunks))

11


In [8]:
for i, chunk in enumerate(chunks):
    print(f"--- Chunk {i+1} ---")
    print(chunk.page_content)
    print("\n" + "="*50 + "\n")

--- Chunk 1 ---
Hamming Code in Computer Network 
Hamming code is a set of error-correction codes that can be used to detect 
and correct the errors that can occur when the data is moved or stored 
from the sender to the receiver. It is technique developed by R.W. 
Hamming for error correction. 
Redundant bits – 
Redundant bits are extra binary bits that are generated and added to the 
information-carrying bits of data transfer to ensure that no bits were lost 
during the data transfer. 
The number of redundant bits can be calculated using the following formula: 
 2^r ≥ m + r + 1 
 where, r = redundant bit, m = data bit
Suppose the number of data bits is 7, then the number of redundant bits can 
be calculated using: 
= 2^4 ≥ 7 + 4 + 1 
Thus, the number of redundant bits= 4 
Parity bits –


--- Chunk 2 ---
be calculated using: 
= 2^4 ≥ 7 + 4 + 1 
Thus, the number of redundant bits= 4 
Parity bits – 
A parity bit is a bit appended to a data of binary bits to ensure that the total 
number

STEP 3: Embeddings

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2"  # or any other open model
)

/tmp/ipython-input-1102338768.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
#FAISS (Facebook AI Similarity Search) is a library for efficient similarity search and clustering of dense vectors.

from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

print(vectorstore)

In [14]:
# Visualize Vector Embeddings

sample_text = "Hamming Code is used for error correction."
sample_embedding = embeddings.embed_query(sample_text)

print(f"Sample text: {sample_text}")
print(f"Embedding dimension: {len(sample_embedding)}")
print(f"Sample embedding (first 10 elements): {sample_embedding[:10]}...")

Sample text: Hamming Code is used for error correction.
Embedding dimension: 384
Sample embedding (first 10 elements): [-0.019811129197478294, 0.07184909284114838, -0.05008786916732788, -0.04794593155384064, -0.05199361965060234, 0.0021552513353526592, 0.01044466532766819, 0.015037737786769867, 0.025476425886154175, 0.03496166318655014]...


### Displaying Metadata of Retrieved Documents

In [40]:
print("Metadata for the top 3 most relevant documents:")

# Retrieve relevant documents using the already defined retriever
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "score_threshold": 0.5,  # try 0.4–0.6
        "k": 3
    }
)

retrieved_docs = retriever.invoke(sample_text)

for i, doc in enumerate(retrieved_docs[:3]):
    print(f"--- Document {i+1} Metadata ---")
    print(doc.metadata)
    print("\n" + "="*50 + "\n")

Metadata for the top 3 most relevant documents:
--- Document 1 Metadata ---
{'producer': 'macOS Version 12.2.1 (Build 21D62) Quartz PDFContext', 'creator': 'Pages', 'creationdate': "D:20260123041704Z00'00'", 'title': 'Hamming Code 2', 'moddate': "D:20260123041704Z00'00'", 'source': 'WINSEM2025-26_VL_CSI2007_00100_ETH_2026-01-23_Hamming-Code.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}




STEP 4: Setting up LLM

In [41]:
# STEP 4: Lightweight LLM

!pip install -q transformers accelerate bitsandbytes

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
import torch

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,
    temperature=0.1, # makes sure that llm sticks to your context only overriding pre-trained knowledge
    do_sample=False,
    return_full_text=False
)

llm = HuggingFacePipeline(pipeline=pipe)

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

In [42]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
You are a helpful assistant.
Answer the question ONLY using the context below.
If the answer is not in the context, say "I don't know".

Context:
{context}

Question:
{question}

Answer:
""",
    input_variables=["context", "question"]
)


In [43]:
def format_docs(docs, max_chars=1500):
    text = "\n\n".join(doc.page_content for doc in docs)
    return text[:max_chars]


In [44]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
)


In [46]:
def safe_rag(query):
    docs = retriever.invoke(query)

    if not docs:
        return "I don't know"

    context = format_docs(docs)

    final_prompt = prompt.format(context=context, question=query)

    return llm.invoke(final_prompt)

In [53]:
query = "Explain hamming code in simple terms"

answer = safe_rag(query)
print(answer)

Both `max_new_tokens` (=200) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Hamming code is a set of error-correction codes that can be used to detect 
and correct the errors that can occur when the data is moved or stored 
from the sender to the receiver. It is developed by R.W. Hamming for error 
correction. Redundant bits are extra binary bits that are generated and 
added to the information-carrying bits of data transfer to ensure that no 
bits were lost during the data transfer. The number of redundant bits can 
be calculated using the formula: 2^r ≥ m + r + 1, where r = redundant bit, 
m = data bit. The number of redundant bits can be calculated using the 
following formula: 
= 2^4 ≥ 7 + 4 + 1, where 4 = redundant bit, 7 = data bit. The number of 
redundant bits is


In [58]:
query = "Who is the most decorated swimmer?"

answer = safe_rag(query)
print(answer)

/usr/local/lib/python3.12/dist-packages/langchain_core/vectorstores/base.py:1048: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='1c7ce121-6ea8-4d7b-b85b-94f31a3ad42e', metadata={'producer': 'macOS Version 12.2.1 (Build 21D62) Quartz PDFContext', 'creator': 'Pages', 'creationdate': "D:20260123041704Z00'00'", 'title': 'Hamming Code 2', 'moddate': "D:20260123041704Z00'00'", 'source': 'WINSEM2025-26_VL_CSI2007_00100_ETH_2026-01-23_Hamming-Code.pdf', 'total_pages': 7, 'page': 6, 'page_label': '7'}, page_content='1101 \nR1------- D3 D5 D7 \nR2--------D3 D6 D7 \nR4--------D5 D6 D7 \n1100110 \n0101   R1 \n1111   R2 \n0011  R4 \nD7 D6 D5 R4 D3 R2 R1\n1 1 0 R4 1 R2 R1'), np.float32(-0.31265175)), (Document(id='de160d0c-5b47-40ca-9f0b-c7a15565128e', metadata={'producer': 'macOS Version 12.2.1 (Build 21D62) Quartz PDFContext', 'creator': 'Pages', 'creationdate': "D:20260123041704Z00'00'", 'title': 'Hamming Code 2', 'moddate': "D:20260123041704Z00'00'", 'source': 'WINSEM2

I don't know


In [60]:
print("\nSOURCES:")
if isinstance(answer, dict) and "source_documents" in answer:
    for doc in answer["source_documents"]:
        print(doc.metadata)
else:
    print("Source documents not available.")


SOURCES:
Source documents not available.
